In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [ ]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

In [ ]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=1)
portfolio

In [4]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=600,
    calculate_features=False,
    estimator=None,
)

In [ ]:
for _ in aac.configure_iter():
    print(aac.get_progress())

In [ ]:
last_model_iter = 0

estimator = None
db = DB()
estimator_pct = 0.5
for _ in aac.configure_iter():
    if aac.get_progress() >= 0.5 and aac.iter - last_model_iter >= 10:
        X, y = get_model_training_data(db)
        estimator = Estimator1(max_cost=TSP_LKH_Solver.MAX_COST, estimator_pct=estimator_pct)
        estimator.fit(X, y)
        estimator.log()
        last_model_iter = aac.iter
        aac.update(estimator=estimator)

In [4]:
# db = DB()
db = DB("database/run-test_923971.db")

In [ ]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

In [ ]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

In [ ]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

In [ ]:
_ = results.loc[results["prefix"].str.startswith("config")]
_.loc[_["surrogate"] == 1]

In [ ]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)